In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [4]:
import pandas as pd
import numpy as np
from faker import Faker
from sqlalchemy import create_engine
#import sql_queries as sql
import configparser
import sys 
import os
import datetime as dt
from tqdm import tqdm

In [5]:
script_dir = os.path.abspath('')
mymodule_dir = os.path.join( script_dir, '..', '..', 'database', 'table_setup' )
sys.path.append( mymodule_dir )

import sql_queries

In [6]:
mymodule_dir = os.path.join( script_dir, '..', '..', 'database', 'connection' )
sys.path.append( mymodule_dir )

import postgres_db

In [7]:
# =============================================================================
# ### SQL CONNECTION
# =============================================================================
encoding = 'utf-16'

config_file_path = os.path.join( script_dir, '..', '..', 'database', 'connection' )
config_file_path += '/config.ini'

db_conn = postgres_db.PostgresConnector(config_file_path=config_file_path,
                                       section='jj_furniture',
                                       encoding=encoding)

In [8]:
fake = Faker()

In [9]:
def get_ids(table_name):
    query = '''
    SELECT {}_id
    FROM {}
    '''

    results = db_conn.queryall(query.format(table_name[:-1], table_name))
    return results[results.columns[0]].values.tolist()

In [19]:
def get_cost(product):
    query = '''
    SELECT cost
    FROM products
    WHERE product_id = {}
    '''
    
    return float(db_conn.queryall(query.format(product))['cost'][0])

In [20]:
print(get_cost(1))

818.24


In [21]:
customer_id_list = get_ids('customers')
product_id_list = get_ids('products')

In [23]:
num_trans = 250000
transaction_id = 1
years = list(range(2010, 2023))
weights = list(range(1, len(years) + 1))
weights = [x/sum(weights) for x in weights]

for trans in tqdm(range(num_trans)):
    
    num_items = np.random.randint(low = 1, high = 5)
    year = np.random.choice(years, size = None, p = weights)
    transaction_date = fake.date_between_dates(date_start = dt.datetime.strptime(str(year) + '-01-01', '%Y-%m-%d'), date_end = dt.datetime.strptime(str(year) + '-12-31', '%Y-%m-%d'))
    customer_id = np.random.choice(customer_id_list).item()
    
    for item in range(num_items): 
        
        product_id = np.random.choice(product_id_list).item()
        line_item_number = item + 1
        sale_or_return = 0
        
        sale_amount = round(get_cost(product_id) * np.random.uniform(low = 1.43, high = 4), 2)
        quantity = np.random.choice([1,2,3], p = [.8, .15, .05]).item()
        
        db_conn.update(sql_queries.insert_transactions_table,
                          (transaction_id,
                          transaction_date,
                          customer_id,
                          product_id,
                          line_item_number,
                          sale_amount,
                          quantity,
                          sale_or_return))
                          
    transaction_id += 1                         

  0%|          | 96/250000 [00:44<32:28:50,  2.14it/s]


KeyboardInterrupt: 